<a href="https://colab.research.google.com/github/juwimana/Trading/blob/main/Stock_Purchase_Valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import packages
import datetime as dt
import pandas as pd
import pandas_datareader.data as data
from pandas_datareader._utils import RemoteDataError
import yfinance as yf
import locale #currency formatting
locale.setlocale(locale.LC_ALL, '')

In [227]:
start_date =dt.datetime(2020,10,1)
end_date = dt.date.today()
stock_data =''

#stock ticker
ticker = 'LTHM'
stock = yf.Ticker(ticker)

In [ ]:
#general stock prices
df_prices = yf.download(ticker, 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
df_prices.head()

In [ ]:
#incomplete url string
str_summary = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'
str_stat = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
str_analysis = 'https://finance.yahoo.com/quote/{}/analysis?p={}'
str_financials= 'https://finance.yahoo.com/quote/{}/financials?p={}'

#complete website url
url_summary = str_summary.format(ticker,ticker)
url_stat = str_stat.format(ticker,ticker)
url_analysis = str_analysis.format(ticker,ticker)
url_financials = str_financials.format(ticker,ticker)

print(url_summary)
print(url_stat)
print(url_analysis)
print(url_financials)

In [ ]:
#read ticker financial summary page
df_summary = pd.read_html(url_summary, header = 0)
df_summary

In [ ]:
summary_2 = df_summary[0]
summary_2

In [ ]:
summary_1 = df_summary[1]
summary_1

In [ ]:
#read ticker financial statistics page
df_stat = pd.read_html(url_stat, header =0)
df_stat

In [ ]:
len(df_stat)

In [ ]:
stat_0 = df_stat[0]
stat_0

In [ ]:
stat_1 = df_stat[1]
stat_1

In [ ]:
stat_5 = df_stat[5]
stat_5

In [ ]:
stat_6 = df_stat[6]
stat_6

In [ ]:
stat_8 = df_stat[8]
stat_8

In [ ]:
#read ticker financial statistics page
df_analysis = pd.read_html(url_analysis, header =0)
df_analysis

In [ ]:
analysis_5 = df_analysis[5]
analysis_5

*   [Operating margin](https://www.investopedia.com/terms/o/operatingmargin.asp) is the profit a company makes on a dollar of sales after paying for variable costs but before paying any interest or taxes
*   [EPS](https://www.investopedia.com/terms/e/eps.asp) indicates how much money a company makes for each share of its stock
*   [Beta](https://www.investopedia.com/terms/b/beta.asp) is a measure of the volatility—or systematic risk—of a security or portfolio compared to the market as a whole
*   [Enterprise Value/EBITDA](https://www.investopedia.com/ask/answers/072715/what-considered-healthy-evebitda.asp#:~:text=The%20enterprise%2Dvalue%2Dto%2D,10%20are%20seen%20as%20healthy.) compares the value of a company—debt included—to the company’s cash earnings less non-cash expenses
*   [Return on equity (ROE)](https://www.investopedia.com/terms/r/returnonequity.asp) measures how effectively management is using a company’s assets to create profits
*   [Debt-to-Equity Ratio (D/E)](https://www.investopedia.com/terms/d/debtequityratio.asp) is a measure of the degree to which a company is financing its operations through debt versus wholly-owned funds

Note: GE stands for Growth Estimates per annum. For example, GE Next Year means growth estimates for next year.



In [240]:
#create stock key metrics DataFrame
stock_metrics= {
    'Company': stock.info.get('shortName'),
    'Ticker': ticker,
    'Sector': stock.info.get('sector'),
    'Market Cap': locale.currency(stock.info.get('marketCap'),grouping=True),
    '52 Week Low': stat_1.loc[3][1],
    '52 Week High': stat_1.loc[2][1],
    'EPS' : summary_1.loc[2][1],
    'Beta' : stock.info.get('beta'),
    'Enterprise Value/EBITDA': stat_0.loc[8][1],
    'Operating Margin': stat_5.loc[0][1],
    'ROE':stat_6.loc[0][1],
    'Debt-to-Equity': stat_8.loc[2][1],
    'GE Next Year': analysis_5.loc[3][1],
    'GE Next 5 Years': analysis_5.loc[4][1],
    'GE Past 5 Years': analysis_5.loc[5][1]
    }
stock_df = pd.DataFrame(data = stock_metrics, index = [0])
stock_df

,Company,Ticker,Sector,Market Cap,52 Week Low,52 Week High,EPS,Beta,Enterprise Value/EBITDA,Operating Margin,ROE,Debt-to-Equity,GE Next Year,GE Next 5 Years,GE Past 5 Years
0,Livent Corporation,LTHM,Basic Materials,"$1,750,636,416.00",3.95,12.29,0.11,None,30.71,8.50%,2.87%,39.46,242.90%,6.15%,NaN
